## Force Field Preparation

1. rearrange atom orders in mol2 file + assign atom type --> correspond to Daniele's g96 file 


In [46]:
import re
import numpy as np
import matplotlib.pyplot as plt
from pylab import *
import os
import scipy.linalg as la
%matplotlib inline  
# maybe some 3D plot for the molecule?? Is it possible?

#ORDER Matrix
# order = [24,25,20,21,22,23,5,15,6,26,27,7,16,29,28,30,31,17,48,49,8,18,51,50,52,53,19,32,54,35,34,13,38,37,36,39,41,42,43
# ,45,47,10,46,9,57,56,14,60,59,58,61,63,64,65,67,69,12,68,11,33,55,40,1,2,44,62,3,4,66,70,71,72,73,...]

In [98]:
class mol: # mol2 file coord container
    
    def __init__(self,ith_mol):
        self.mol_i = ith_mol
        self.cord = np.zeros(N_atoms*3)
    
class g96:   # g96 (gromacs structure file) container
    
    def __init__(self):
        self.type = []
        self.cord = np.zeros(N_atoms*3)
        
class itp:
    
    def __init__(self):
        
        ##for [atomtypes] block:
        self.type = []  # only type --> length < N_atoms
        # sigma unit: nm in Gromacs --> A in CP2K (note: in CP2K, list sigma/2). So 10*sigma/2 for conversion
        self.sigma = np.zeros(N_atoms)
        # eps. unit: kJ/mol (in Gromacs) --> kcal/mol in CP2K. so epsilon/4.184 for conversion from Gromacs to CP2K
        self.epsilon = np.zeros(N_atoms) 
        #self.charge   #not store charge--> always 0.000
        
        ##for [atoms] block:
        self.RES = ''
        self.atoms = [] #length = N_atoms    #C/N/...
        self.atomtype = [] #length = N_atoms #C1/N1/...
        #self.CHARGE = False
        self.charges = np.zeros(N_atoms)
        self.mass = np.zeros(N_atoms)
        
        ##for [bonds] block:
        self.bond1 = []
        self.bond2 = []
        self.r_eq = []  # equilibrium bond length (unit: nm)
        self.k_b = []   # bond force const (unit: kJ/mol-nm^2)
        
        ##for [angles] block:
        self.angle1 = []
        self.angle2 = []
        self.angle3 = []
        self.a_eq = []
        self.k_a = []
        
        ##for [dihedrals]block:
        #proper 
        self.d1p = []
        self.d2p = []
        self.d3p = []
        self.d4p = []
        self.p_eq = []
        self.k_p = []
        #improper
        self.d1i = []
        self.d2i = []
        self.d3i = []
        self.d4i = []
        self.im_eq = []
        self.k_im = []

class CoordParser:   # Parse mol2/g96/... files
    
    def __init__(self, path, order):
        self.path = path
        #orderM --> order matrix. can we get this automatically???
        self.orderM = [x -1 for x in order]
        self.mol_p = ''
        self.g96_p = ''
        self.itp_p = ''
        self.mol = []
        self.g96 = []
        self.itp = []
        
        ##unit conversion from Gromacs (nm/kJ-mol^-1/...) to CP2K/CHARMM prm (A/kcal-mol^-1/...)
        self.nm2A = 10. #nm --> A
        self.k_b_g2c = 0.00119502868 # (kJ/mol-nm^2) --> (1/2)*(kcal/mol-A^2): (1/4.184)*(1/100)*(1/2)
        self.k_a_g2c = 0.11950286806 # (kJ/mol) --> (1/2)*(kcal/mol): (1/4.184)*(1/2)
        
    def CheckFiles(self):
        
        count_mol = 0
        count_g96 = 0
        count_itp = 0

        for fname in os.listdir(self.path):
            if fname.endswith('.mol2') and count_mol == 0:
                self.mol_p = fname
                count_mol = count_mol + 1
            elif fname.endswith('.mol2') and count_mol != 0:
                count_mol = count_mol + 1
            elif fname.endswith('.g96') and count_g96 == 0:
                self.g96_p = fname
                count_g96 = count_g96 + 1
            elif fname.endswith('.g96') and count_g96 != 0:
                count_g96 = count_g96 + 1
            # read in itp (gromacs topology file) --> use to convert to CP2K (or other) prm/psf format
            elif fname.endswith('.itp') and count_itp == 0:
                self.itp_p = fname
                count_itp = count_itp + 1
            elif fname.endswith('.itp') and count_itp != 0:
                count_itp = count_itp + 1
            
                
        if (count_mol > 1):
            print('Warning: More than 1 mol2 file')
        if (count_g96 > 1):
            print('Warning: More than 1 g96 file')
        if (count_itp > 1):
            print('Warning: More than 1 itp file')
        print(self.mol_p+' is in use for mol2 file')
        print(self.g96_p+' is in use for g96 file')
        print(self.itp_p+' is in use for itp file')
                            
    def R_mol2Coord(self):

        if not self.mol_p:
            print('No mol2 file assigned')
            return
        
        print('Use '+self.mol_p)
        with open (self.path+self.mol_p) as file:
            lines = file.readlines()
            for num, line in enumerate(lines):
                if 'ATOM' in line:
                    line_s = num+1
                    break
            if line_s is None:
                print('MOL2 file has no atom coordinates...')
                return
            for i in range(N_mol):
                self.mol.append(mol(i))
                for j in range(N_atoms):
                    self.mol[i].cord[3*j] = lines[line_s+i*N_atoms+j].split()[2]   #unit in A
                    self.mol[i].cord[3*j+1] = lines[line_s+i*N_atoms+j].split()[3]
                    self.mol[i].cord[3*j+2] = lines[line_s+i*N_atoms+j].split()[4]
        #for i in range(N_mol):
        #    print(self.mol[i].cord)
            
    def R_grog96(self):
        
        self.CheckFiles()
        
        if not self.g96_p:
            print('no g96 file assigned')
            return
        
        print('Use '+self.g96_p)
        with open (self.path+self.g96_p) as file:
            lines = file.readlines()
            for num, line in enumerate(lines):
                if 'POSITION' in line:
                    line_s = num+1
                    break
            if line_s is None:
                print('g96 file has no position info...')
                return
            Nmol_g96 = 1  # prefer to set Nmol_g96=1 always... get single mol atom type is enough
            for i in range(Nmol_g96): 
                self.g96.append(g96())
                #print(i)
                for j in range(N_atoms):
                    self.g96[i].type.append(lines[line_s+i*N_atoms+j].split()[2])
                    self.g96[i].cord[3*j] = lines[line_s+i*N_atoms+j].split()[4]  #unit in nm
                    self.g96[i].cord[3*j+1] = lines[line_s+i*N_atoms+j].split()[5]
                    self.g96[i].cord[3*j+2] = lines[line_s+i*N_atoms+j].split()[6]
        #print(self.g96[0].type)
        #print(self.g96[0].cord)
        
    def W_mol2tog96(self):  # from structural file mol2, build Gromacs structural file g96
        
        self.CheckFiles()
        self.R_mol2Coord()
        self.R_grog96()
        
        L_shift = 0.  # shift along axis...
        
        with open (self.path+'mol2g96'+'.g96'+'_new', 'w') as out:
            out.write("TITLE\n")
            out.write("mol2tog96\n")
            out.write("END\n")
            out.write("POSITION\n")
            for i in range(N_mol):
                for j in range(N_atoms):
                    out.write('    '+str(4)+' RES    '+str(self.g96[0].type[j])+'    '+str(3*N_atoms+j+1)+'  '\
                              +str(L_shift + (self.mol[i].cord[3*self.orderM[j]])/10.)+'  '+\
                              str(L_shift + (self.mol[i].cord[3*self.orderM[j]+1])/10.)+\
                             '    '+str(L_shift + (self.mol[i].cord[3*self.orderM[j]+2])/10.)) #need modify on Ord[j]...
                    out.write('\n')
                    #print(str(i))
                    #print(str(self.g96[0].type[j]))
            
            out.write("END\n")
            out.write("BOX\n")
            out.write('    '+'0.000000000'+'    '+'0.000000000'+'    '+'0.000000000\n')
            out.write("END\n")
            
    def R_itp(self): # read Gromacs itp: mainly for Nonbonded part in prm and for building psf
        
        if not self.itp_p:
            print('No itp file assigned')
            return
        
        print('Use '+self.itp_p)
        with open (self.path+self.itp_p) as file:
            lines = file.readlines()
            for num, line in enumerate(lines):
                if '[ atomtypes ]' in line:
                    line_s = num+2
                elif '[ atoms ]' in line:
                    line_a = num+2
                elif '[ bonds ]' in line:
                    line_b = num+2
                elif '[ angles ]' in line:
                    line_ang = num+2
                elif '[ dihedrals ]' in line:
                    line_d = num+2
                elif '[ pairs ]' in line:
                    line_p = num+2
                    break
            if line_s is None:
                print('itp file has no atomtypes block...')
                return
            elif line_a is None:
                print('itp file has no [atoms] block')
                return

            # for loop: for [atomtypes] atom type/sigma/epsilon in itp
            Nmol_g96 = 1  # itp seems to be for single molecule is enough...(??)
            for i in range(Nmol_g96):
                self.itp.append(itp())
                for j in range(line_s-line_s,line_a-7-line_s,1): # this loop for [atomtypes] block
                    if lines[line_s+i*N_atoms+j] == ' \n':
                        break
                    self.itp[i].type.append(lines[line_s+i*N_atoms+j].split()[0])
                    self.itp[i].sigma[j] = float(lines[line_s+i*N_atoms+j].split()[4])
                    self.itp[i].epsilon[j] = float(lines[line_s+i*N_atoms+j].split()[5])            
                for j in range(N_atoms): # this loop for [atoms] [bonds] block
                    if j == 0:
                        self.itp[i].RES = lines[line_a].split()[3]
                    self.itp[i].atomtype.append(lines[line_a+i*N_atoms+j].split()[1])
                    self.itp[i].atoms.append(str(self.itp[i].atomtype[j])[0])
                    self.itp[i].charges[j] = lines[line_a+i*N_atoms+j].split()[6]
                    self.itp[i].mass[j] = lines[line_a+i*N_atoms+j].split()[7]
                for j in range(line_b, line_ang, 1):
                    if lines[i*N_atoms+j] == ' \n':
                        break
                    elif lines[i*N_atoms+j].split()[0] == ';':
                        continue
                    self.itp[i].bond1.append(lines[i*N_atoms+j].split()[0]) 
                    self.itp[i].bond2.append(lines[i*N_atoms+j].split()[1])
                    self.itp[i].r_eq.append(lines[i*N_atoms+j].split()[3])
                    self.itp[i].k_b.append(lines[i*N_atoms+j].split()[4])
                for j in range(line_ang, line_d, 1):
                    if lines[i*N_atoms+j] == ' \n':
                        break
                    elif lines[i*N_atoms+j].split()[0] == ';':
                        continue
                    self.itp[i].angle1.append(lines[i*N_atoms+j].split()[0]) 
                    self.itp[i].angle2.append(lines[i*N_atoms+j].split()[1])
                    self.itp[i].angle3.append(lines[i*N_atoms+j].split()[2])
                    self.itp[i].a_eq.append(lines[i*N_atoms+j].split()[4])
                    self.itp[i].k_a.append(lines[i*N_atoms+j].split()[5])
                for j in range(line_d, line_p, 1):
                    if lines[i*N_atoms+j] == ' \n':
                        break
                    ## deal with special case... (??? correct ???) should I have to repeat this dihedral six times??
                    elif lines[i*N_atoms+j].split()[0] == ';' and lines[i*N_atoms+j].split()[1] == 'delta1':
                        self.itp[i].d1p.append(lines[i*N_atoms+j+1].split()[0]) 
                        self.itp[i].d2p.append(lines[i*N_atoms+j+1].split()[1])
                        self.itp[i].d3p.append(lines[i*N_atoms+j+1].split()[2])
                        self.itp[i].d4p.append(lines[i*N_atoms+j+1].split()[3])
                        self.itp[i].d1p.append(lines[i*N_atoms+j+7].split()[0]) 
                        self.itp[i].d2p.append(lines[i*N_atoms+j+7].split()[1])
                        self.itp[i].d3p.append(lines[i*N_atoms+j+7].split()[2])
                        self.itp[i].d4p.append(lines[i*N_atoms+j+7].split()[3])
                        break
                    elif lines[i*N_atoms+j].split()[0] == ';':
                        continue
                    if lines[i*N_atoms+j].split()[4] == '2':
                        self.itp[i].d1i.append(lines[i*N_atoms+j].split()[0]) 
                        self.itp[i].d2i.append(lines[i*N_atoms+j].split()[1])
                        self.itp[i].d3i.append(lines[i*N_atoms+j].split()[2])
                        self.itp[i].d4i.append(lines[i*N_atoms+j].split()[3])
                        self.itp[i].im_eq.append(lines[i*N_atoms+j].split()[5])
                        self.itp[i].k_im.append(lines[i*N_atoms+j].split()[6])
                    elif lines[i*N_atoms+j].split()[4] == '1':
                        self.itp[i].d1p.append(lines[i*N_atoms+j].split()[0]) 
                        self.itp[i].d2p.append(lines[i*N_atoms+j].split()[1])
                        self.itp[i].d3p.append(lines[i*N_atoms+j].split()[2])
                        self.itp[i].d4p.append(lines[i*N_atoms+j].split()[3])
                                        
            #print(self.itp[0].d1p)
            #print(self.itp[0].d2p)
            #print(self.itp[0].d3p)
            #print(self.itp[0].d4p)
            #print(self.itp[0].d1i)
            #print(self.itp[0].d2i)
            #print(self.itp[0].d3i)
            #print(self.itp[0].d4i)
    
    def W_itp2prmBADI(self): # bond/angle/dihedrals/improper sections of prm from Gromacs itp
        self.CheckFiles()
        self.R_itp()
        with open (self.path+'itp2prm.prm','w') as out:
            ##BONDS
            out.write('BONDS\n')
            for i in range(len(self.itp[0].bond1)):
                out.write(self.itp[0].atomtype[int(self.itp[0].bond1[i])-1]+' '+\
                          self.itp[0].atomtype[int(self.itp[0].bond2[i])-1]+' '+\
                          str(float("{:3.6f}".format(float(self.itp[0].k_b[i])*self.k_b_g2c)))+' '+\
                          str(float("{:2.6f}".format(float(self.itp[0].r_eq[i])*self.nm2A))) +'\n')
            ##ANGLES
            out.write('ANGLES\n')
            for i in range(len(self.itp[0].angle1)):
                out.write(self.itp[0].atomtype[int(self.itp[0].angle1[i])-1]+' '+\
                          self.itp[0].atomtype[int(self.itp[0].angle2[i])-1]+' '+\
                          self.itp[0].atomtype[int(self.itp[0].angle3[i])-1]+' '+\
                          str(float("{:2.6f}".format(float(self.itp[0].k_a[i])*self.k_a_g2c)))+' '+\
                          str(float("{:3.7f}".format(float(self.itp[0].a_eq[i]))))+'\n')
            ##DIHEDRALS
            out.write('DIHEDRALS\n')
#            for i in range(len(self.itp[0].d1p)):
#                out.write(self.itp[0].atomtype[int(self.itp[0].d1p[i])-1]+' '+\
#                         self.itp[0].atomtype[int(self.itp[0].d2p[i])-1]+' '+\
#                         self.itp[0].atomtype[int(self.itp[0].d3p[i])-1]+' '+\
#                         self.itp[0].atomtype[int(self.itp[0].d4p[i])-1]+' '+
#                          '\n'\)
            out.write('\n')
#Not deal with dihedrals for now...
            out.write('IMPROPER\n')
            for i in range(len(self.itp[0].d1i)):
                out.write(self.itp[0].atomtype[int(self.itp[0].d1i[i])-1]+' '+\
                         self.itp[0].atomtype[int(self.itp[0].d2i[i])-1]+' '+\
                         self.itp[0].atomtype[int(self.itp[0].d3i[i])-1]+' '+\
                         self.itp[0].atomtype[int(self.itp[0].d4i[i])-1]+' '+
                         str(float("{:2.6f}".format(float(self.itp[0].k_im[i])*self.k_a_g2c))) + ' '+\
                          str(0)+' '+ str(float("{:3.7f}".format(float(self.itp[0].im_eq[i])))) + '\n')                

    def W_itp2prmNB(self): # build NonBonded section of prm from GROMACS itp
        self.CheckFiles()
        self.R_itp()
        fac_ = 0.5**(1/6)
        with open (self.path+'itp2prmNB'+'.prm', 'w') as out:
            out.write("NONBONDED\n")
            for num,line in enumerate(self.itp[0].type):
                out.write(line+'    0.000000    ')
                # epsilon in kJ/mol (Gromacs) --> kcal/mol (CP2K). ALso note, in CHARMM format, epsilon is negative
                # sigma (Rmin in CP2K) nm --> A. 
                out.write(str(self.itp[0].epsilon[num]/(-4.184))+'    ')  
                out.write(str(5*self.itp[0].sigma[num]/fac_)+'    ')
                out.write('0.000000'+'    '+'0.00'+'    '+'0.000'+'\n')   # factor 5???
            out.write('END\n')
                        
    def W_itp2psf(self): # build psf from GROMACS ipt
        
        self.CheckFiles()
        self.R_itp()
        
        with open (self.path+'itp2psf.psf','w') as out:
            out.write('PSF\n')
            out.write('\n')
            out.write('{:>10} {:>8}'.format('1','!NTITLE') + '\n')
            out.write(' '+self.itp[0].RES+'\n')
            out.write('\n')
            out.write('{:>10} {:>7} '.format(str(len(self.itp[0].atoms)),'!NATOM') + '\n')
            for j in range(N_atoms):
                out.write('{:>10} {:>4} {:>14} {:>4} {:>7} {:>9} {:>14} {:>14} {:>12}'.format(str(j+1),\
                            self.itp[0].RES,str(1),self.itp[0].RES,self.itp[0].atoms[j],self.itp[0].atomtype[j],\
                            self.itp[0].charges[j],self.itp[0].mass[j],str(0)) + '\n')
            out.write('\n')
            out.write('{:>10} {:>7} '.format(str(len(self.itp[0].bond1)),'!NBOND') + '\n')
            jj_ = len(self.itp[0].bond1) % 4
            kk_ = len(self.itp[0].bond1)//4
            for j in range(kk_):
                out.write('{:>10} {:>10} {:>10} {:>10} {:>10} {:>10} {:>10} {:>10}'.format(\
                            self.itp[0].bond1[4*j],self.itp[0].bond2[4*j],\
                            self.itp[0].bond1[4*j+1],self.itp[0].bond2[4*j+1],\
                            self.itp[0].bond1[4*j+2],self.itp[0].bond2[4*j+2],\
                            self.itp[0].bond1[4*j+3],self.itp[0].bond2[4*j+3]) + '\n')
            if jj_ != 0:
                if jj_ == 1:
                    out.write('{:>10} {:>10}'.format(self.itp[0].bond1[4*kk_],self.itp[0].bond2[4*kk_]))
                elif jj_ == 2:
                    out.write('{:>10} {:>10} {:>10} {:>10}'.format(self.itp[0].bond1[4*kk_],self.itp[0].bond2[4*kk_],\
                            self.itp[0].bond1[4*kk_+1],self.itp[0].bond2[4*kk_+1]))
                elif jj_ == 3:
                    out.write('{:>10} {:>10} {:>10} {:>10} {:>10} {:>10}'.format(\
                                self.itp[0].bond1[4*kk_],self.itp[0].bond2[4*kk_],\
                                self.itp[0].bond1[4*kk_+1],self.itp[0].bond2[4*kk_+1],\
                                self.itp[0].bond1[4*kk_+2],self.itp[0].bond2[4*kk_+2]))
                out.write('\n')
            out.write('\n')
            out.write('{:>10} {:>8} '.format(str(len(self.itp[0].angle1)),'!NTHETA') + '\n')
            jj_ = len(self.itp[0].angle1) % 3
            kk_ = len(self.itp[0].angle1)//3
            for j in range(kk_):
                out.write('{:>10} {:>10} {:>10} {:>10} {:>10} {:>10} {:>10} {:>10} {:>10}'.format(\
                            self.itp[0].angle1[3*j],self.itp[0].angle2[3*j],self.itp[0].angle3[3*j],\
                            self.itp[0].angle1[3*j+1],self.itp[0].angle2[3*j+1],self.itp[0].angle3[3*j+1],\
                            self.itp[0].angle1[3*j+2],self.itp[0].angle2[3*j+2],self.itp[0].angle3[3*j+2])\
                            + '\n')
            if jj_ != 0:
                if jj_ == 1:
                    out.write('{:>10} {:>10} {:>10}'.format(self.itp[0].angle1[3*kk_],\
                                self.itp[0].angle2[3*kk_],self.itp[0].angle3[3*kk_]))
                if jj_ == 2:
                    out.write('{:>10} {:>10} {:>10} {:>10} {:>10} {:>10}'.format(self.itp[0].angle1[3*kk_],\
                                self.itp[0].angle2[3*kk_],self.itp[0].angle3[3*kk_],\
                                self.itp[0].angle1[3*kk_+1],self.itp[0].angle2[3*kk_+1],self.itp[0].angle3[3*kk_+1]))
                out.write('\n')
            out.write('\n')
            out.write('{:>10} {:>6} '.format(str(len(self.itp[0].d1p)),'!NPHI') + '\n')
            jj_ = len(self.itp[0].d1p) % 2
            kk_ = len(self.itp[0].d1p)//2
            for j in range(kk_):
                out.write('{:>10} {:>10} {:>10} {:>10} {:>10} {:>10} {:>10} {:>10} '.format(\
                            self.itp[0].d1p[2*j],self.itp[0].d2p[2*j],self.itp[0].d3p[2*j],self.itp[0].d4p[2*j],\
                            self.itp[0].d1p[2*j+1],self.itp[0].d2p[2*j+1],self.itp[0].d3p[2*j+1],self.itp[0].d4p[2*j+1])\
                            + '\n')
            if jj_ != 0:
                out.write('{:>10} {:>10} {:>10} {:>10}'.format(self.itp[0].d1p[2*kk_],\
                            self.itp[0].d2p[2*kk_],self.itp[0].d3p[2*kk_],self.itp[0].d4p[2*kk_]))
                out.write('\n')
            out.write('\n')
            out.write('{:>10} {:>8} '.format(str(len(self.itp[0].d1i)),'!NIMPHI') + '\n')
            jj_ = len(self.itp[0].d1i) % 2
            kk_ = len(self.itp[0].d1i)//2
            for j in range(kk_):
                out.write('{:>10} {:>10} {:>10} {:>10} {:>10} {:>10} {:>10} {:>10} '.format(\
                            self.itp[0].d1i[2*j],self.itp[0].d2i[2*j],self.itp[0].d3i[2*j],self.itp[0].d4i[2*j],\
                            self.itp[0].d1i[2*j+1],self.itp[0].d2i[2*j+1],self.itp[0].d3i[2*j+1],self.itp[0].d4i[2*j+1])\
                            + '\n')
            if jj_ != 0:
                out.write('{:>10} {:>10} {:>10} {:>10}'.format(self.itp[0].d1i[2*kk_],\
                            self.itp[0].d2i[2*kk_],self.itp[0].d3i[2*kk_],self.itp[0].d4i[2*kk_]))
                out.write('\n')
            out.write('\n')
            out.write('{:>10} '.format(str(0)) + ' !NDON' + '\n')
            out.write('\n')
            out.write('{:>10} '.format(str(0)) + ' !NACC' + '\n')
            out.write('\n')
            out.write('{:>10} '.format(str(0)) + ' !NNB' + '\n')
            out.write('\n')
            out.write('{:>10} '.format(str(0)) + ' !NGRP' + '\n')
            out.write('\n')

            
    def W_g96toxyz(self):
        
        self.CheckFiles()
        self.R_grog96()
        
        with open (self.path+'g96toxyz.xyz', 'w') as out:
            out.write('    '+str(len(self.g96[0].type))+'\n')
            out.write('\n')
            for j in range(len(self.g96[0].type)):
                out.write(' '+(self.g96[0].type[j])[0]+'    '+str(10*self.g96[0].cord[3*j])+'    '\
                         +str(10*self.g96[0].cord[3*j+1])+'    '+str(10*self.g96[0].cord[3*j+2])+'\n')
            out.write('\n')
    
    # most prilimary shift function...
    def W_g96toxyz_shift(self):
        
        self.CheckFiles()
        self.R_grog96()
        
        shift_par = 50.
        
        with open (self.path+'g96toxyz_shift.xyz', 'w') as out:
            out.write('    '+str(len(self.g96[0].type))+'\n')
            out.write('\n')
            for j in range(len(self.g96[0].type)):
                out.write(' '+(self.g96[0].type[j])[0]+'    '+str(10*self.g96[0].cord[3*j]+shift_par)+'    '\
                         +str(10*self.g96[0].cord[3*j+1]+shift_par)+'    '\
                          +str(10*self.g96[0].cord[3*j+2]+shift_par)+'\n')
            out.write('\n')
            
    def W_CP2Kin_KIND(self):
        
        self.CheckFiles()
        self.R_itp()
    
        with open (self.path+'CP2Kin_KIND.dat', 'w') as out:
            for i in range(len(self.itp[0].type)):
                out.write('  &KIND '+self.itp[0].type[i]+'\n')
                out.write('   ELEMENT '+(self.itp[0].type[i])[0]+'\n')
                out.write('  &END KIND\n')
                
    def W_index(self):
        
        with open (self.path+'index.ndx', 'w') as out:
            jj_ = N_atoms // 15
            kk_ = N_atoms % 15
            for i in range(jj_):
                out.write('{:>4} {:>5} {:>5} {:>5} {:>5} {:>5} {:>5} {:>5} {:>5} {:>5} {:>5} {:>5} {:>5} {:>5} {:>5}'.format(\
                                        str(i*15+1), str(i*15+2), str(i*15+3), str(i*15+4), str(i*15+5), \
                                       str(i*15+6), str(i*15+7), str(i*15+8), str(i*15+9), str(i*15+10), \
                                       str(i*15+11), str(i*15+12), str(i*15+13), str(i*15+14), str(i*15+15)))
                out.write('\n')
            if kk_ != 0:
                out.write('{:>4} '.format(str(jj_*15+1)))
                for i in range(kk_-1):
                    out.write('{:>5} '.format(str(jj_*15+i+2)))
                    
    
    

In [100]:
path = './test_crystal/111/m4/'
N_atoms = 85
N_mol = 1

# note: order is mol2 to g96... i.e. 24th atom in mol2 is the 1st atom in g96
# note: so the 5th atom in mol2 which is a N atom, is the 7th atom in g96...
order = [24,25,20,21,22,23,5,15,6,26,27,7,16,29,28,30,31,17,48,49,8,18,51,50,52,53,19,32,54,35,34,13,38,37,36,39,\
         41,42,43,45,47,10,46,9,57,56,14,60,59,58,61,63,64,65,67,69,12,68,11,33,55,40,1,2,44,62,3,4,66,70,71,72,73\
         ,74,75,76,77,78,79,80,81,82,83,84,85]

crystal = CoordParser(path,order)
#crystal.W_mol2tog96()
crystal.W_g96toxyz()

Y6_111_m4.mol2 is in use for mol2 file
y6_opt_freq_min.g96 is in use for g96 file
 is in use for itp file
Y6_111_m4.mol2 is in use for mol2 file
y6_opt_freq_min.g96 is in use for g96 file
 is in use for itp file
Use y6_opt_freq_min.g96


In [45]:
path = './test/'
N_atoms = 85
N_mol = 2

# note: order is mol2 to g96... i.e. 24th atom in mol2 is the 1st atom in g96
# note: so the 5th atom in mol2 which is a N atom, is the 7th atom in g96...
order = [24,25,20,21,22,23,5,15,6,26,27,7,16,29,28,30,31,17,48,49,8,18,51,50,52,53,19,32,54,35,34,13,38,37,36,39,\
         41,42,43,45,47,10,46,9,57,56,14,60,59,58,61,63,64,65,67,69,12,68,11,33,55,40,1,2,44,62,3,4,66,70,71,72,73\
         ,74,75,76,77,78,79,80,81,82,83,84,85]

BOX = [14.46910,20.95814,28.68215,0.0,0.0,-3.09819,0.0,-3.36521,-10.82881]
# Unit cell vector for Y6 111 crystal structure...
#BOX in gromacs format (gro/g96...) --> v1(x) v2(y) v3(z) v1(y) v1(z) v2(x) v2(z) v3(x) v3(y)
#--> note: the last 6 values are optional, and Gromacs only supports v1(y) = v1(z) = v2(z) = 0 

test = CoordParser(path,order)

## Read functions: read mol2/g96/itp files, check what files are in current directory...
#test.CheckFiles()
##test.R_mol2Coord()
#test.R_grog96()
#test.R_itp()

## Write functions: 
## 1. from mol2 file, build gromacs g96 file 
## 2. from gromacs itp file, build (part of) prm file 3. from gromacs itp file, build psf
#test.W_mol2tog96()
test.W_itp2prmNB()
#test.W_itp2psf()
#test.W_g96toxyz()
#test.W_CP2Kin_KIND()
#test.W_itp2prmBADI()
#test.W_g96toxyz_shift()


JMCC20_Y6_dimer.mol2 is in use for mol2 file
y6_opt_freq_min.g96 is in use for g96 file
Y6_HTdelta_modified.itp is in use for itp file
Use Y6_HTdelta_modified.itp


In [65]:
path = './test_dimer/m2/'
N_atoms = 85
N_mol = 1

# note: order is mol2 to g96... i.e. 24th atom in mol2 is the 1st atom in g96
# note: so the 5th atom in mol2 which is a N atom, is the 7th atom in g96...
order = [24,25,20,21,22,23,5,15,6,26,27,7,16,29,28,30,31,17,48,49,8,18,51,50,52,53,19,32,54,35,34,13,38,37,36,39,\
         41,42,43,45,47,10,46,9,57,56,14,60,59,58,61,63,64,65,67,69,12,68,11,33,55,40,1,2,44,62,3,4,66,70,71,72,73\
         ,74,75,76,77,78,79,80,81,82,83,84,85]

dimer = CoordParser(path,order)
#dimer.R_mol2Coord()
dimer.W_mol2tog96()
#dimer.W_g96toxyz()

Y6_m2.mol2 is in use for mol2 file
y6_opt_freq_min.g96 is in use for g96 file
 is in use for itp file
Use Y6_m2.mol2
Y6_m2.mol2 is in use for mol2 file
y6_opt_freq_min.g96 is in use for g96 file
 is in use for itp file
Use y6_opt_freq_min.g96


In [113]:
# for xyz file with multiple molecules, each molecule has to arrange its atoms in a specified order
def W_xyzinOrder(path,N_a,pathw): #rearrange each molecule in the correct atoms order
    
    with open (path) as file:
        lines = file.readlines()
        tot_a = int(lines[0].split()[0])
        tot_m = int(tot_a/N_a)
        
        with open (pathw,'w') as out:
        
            out.write(lines[0])
            out.write('\n')
        
            for i in range(tot_m):
                for j in range(N_a):
                    out.write(lines[1+order[j]+i*85])
    
    

In [114]:
path = './test_crystal/10x2x1and2/Y6_crystal_10x2x1.xyz'
N_atoms = 85
pathw = './test_crystal/10x2x1and2/Y6_crystal_10x2x1.xyz-order'

# note: order is mol2 to g96... i.e. 24th atom in mol2 is the 1st atom in g96
# note: so the 5th atom in mol2 which is a N atom, is the 7th atom in g96...
order = [24,25,20,21,22,23,5,15,6,26,27,7,16,29,28,30,31,17,48,49,8,18,51,50,52,53,19,32,54,35,34,13,38,37,36,39,\
         41,42,43,45,47,10,46,9,57,56,14,60,59,58,61,63,64,65,67,69,12,68,11,33,55,40,1,2,44,62,3,4,66,70,71,72,73\
         ,74,75,76,77,78,79,80,81,82,83,84,85]
W_xyzinOrder(path,N_atoms,pathw)


## CP2K FIST single point energy parser

In [27]:
class cp2kSP:
    
    def __init__(self, path):
        self.bond = 0.
        self.angle = 0.
        self.torsion = 0.
        self.imptor = 0.
        self.bonded = 0. # kcal/mol
        self.totE = 0. # au
        self.NB = 0. # kJ/mol
        self.au2kcal = 627.5095 #au --> kcal/mol
        self.kcal2kj = 4.184  #kcal/mol --> au
        self.NB_kJ = 0.
        self.path = path
        
    def CP2Kparser(self):
        
        with open(self.path+'sp.log') as file:
            lines = file.readlines()
            for num, line in enumerate(lines):
                if 'FIST energy contributions' in line:
                    line_b = num+1
                elif 'ENERGY|' in line:
                    line_t = num
                    break
            self.bond = float(lines[line_b].split()[2])
            self.angle = float(lines[line_b].split()[5])
            self.torsion = float(lines[line_b+1].split()[2])
            self.imptor = float(lines[line_b+1].split()[5])
            self.bonded = self.bond + self.angle + self.torsion + self.imptor
            self.totE = float(lines[line_t].split()[8])
            self.NB = self.totE*self.au2kcal*self.kcal2kj - self.bonded*self.kcal2kj
        print(str(self.NB)+' kJ/mol')
        

In [28]:
path = './test/'

test = cp2kSP(path)
test.CP2Kparser()

-355.7898519089881 kJ/mol


## Nonbonded interactions calculators: Electrostatics and LJ

In [38]:
## Electrostatic interation
eps0 = 8.854187817*10**(-12)  # C^2/J-m
qq = 1.60217733*10**(-19)  # C
Avog = 6.02214076*10**23  # Avogadro number
A2m = 10**(-10)  # ang to meter
J2kJpermol = Avog/1000.

def calcElec(r_ij, charge1, charge2):
    EElec = (charge1*charge2*(qq**2)/(4*math.pi*eps0*r_ij*A2m))*J2kJpermol
    print(EElec)

In [42]:
# C2 - S8
r_ij = 3.808
charge1 = 0.255869
charge2 = -0.180426

calcElec(r_ij, charge1,charge2)

# O32 - O47
r_ij = 12.903
charge1 = -0.476438
charge2 = -0.476438

calcElec(r_ij, charge1,charge2)

-16.843538213152115
24.441933691283236


In [ ]:
def LJ_charmm(eps, sig):
    

# LJ
####INPUT####
#ATOM1
eps1 = 
sig1 =
#ATOM2
eps2 =
sig2 =
parm = ''  #CHM(CHARMM)/GRO(GROMACs)
#############

#combination rule:
eps_ = (eps1*eps2)**(0.5)
sig_ = (sig1+sig2)*0.5

if parm = 'CHM':
    pass
elif parm = 'GRO':  # conversion: units, factors, ...
    eps_ = 
    sig_ = 
else:
    print('no such force field')
    

